# Training a tabular fast.ai NN for stock signal generation

## on a Technical Indicators and OHCV History

In [9]:
from fastai.tabular import *
from tqdm import tqdm_notebook as tqdm

In [10]:
import pandas as pd
from TechnicalIndicators import sma, ema, macd, bollinger_bands, obv

### Download backtesting data

In [11]:
rawdf = pd.read_csv('../hismindata_stock/NASDAQ/AAPL.csv')
rawdf.head()

,date,minute,label,high,low,average,volume,notional,numberOfTrades,marketHigh,...,marketAverage,marketVolume,marketNotional,marketNumberOfTrades,open,close,marketOpen,marketClose,changeOverTime,marketChangeOverTime
0,20190102,09:30,09:30 AM,155.190,154.58,154.841,3223,499052.46,35,155.25,...,154.789,1107615,1.714463e+08,2735,154.740,154.78,154.890,154.780,0.000000,0.000000
1,20190102,09:31,09:31 AM,155.520,154.78,155.197,1674,259800.48,20,155.55,...,155.232,158277,2.456971e+07,1018,154.780,155.37,154.790,155.160,0.002299,0.002862
2,20190102,09:32,09:32 AM,155.250,154.80,155.107,3153,489051.70,32,155.30,...,155.007,155007,2.402724e+07,1144,155.170,154.85,155.165,154.807,0.001718,0.001408
3,20190102,09:33,09:33 AM,154.870,154.60,154.669,5104,789430.81,48,154.91,...,154.705,170003,2.630039e+07,1264,154.805,154.60,154.830,154.670,-0.001111,-0.000543
4,20190102,09:34,09:34 AM,154.975,154.65,154.826,2948,456427.22,22,155.00,...,154.809,146408,2.266528e+07,1099,154.650,154.72,154.630,154.750,-0.000097,0.000129


In [6]:
dataset_df = rawdf[['date', 'minute', 'label', 'open', 'high', 'low', 'close', 'volume' ]]
print(dataset_df.shape)
dataset_df.head()

(28470, 8)


,date,minute,label,open,high,low,close,volume
0,20190102,09:30,09:30 AM,154.740,155.190,154.58,154.78,3223
1,20190102,09:31,09:31 AM,154.780,155.520,154.78,155.37,1674
2,20190102,09:32,09:32 AM,155.170,155.250,154.80,154.85,3153
3,20190102,09:33,09:33 AM,154.805,154.870,154.60,154.60,5104
4,20190102,09:34,09:34 AM,154.650,154.975,154.65,154.72,2948


In [8]:
# Convert date, minute, and label columns into one -> time = timestamp
timestamp_df = pd.DataFrame(columns=['time'])

for index, row in dataset_df.iterrows():
    f = str(row.date) + row.minute + row.label
    timestamp = datetime.strptime(f, '')
    
dataset_df['time'] = timestamp_df

2019010209:3009:30 AM
2019010209:3109:31 AM
2019010209:3209:32 AM
2019010209:3309:33 AM
2019010209:3409:34 AM
2019010209:3509:35 AM
2019010209:3609:36 AM
2019010209:3709:37 AM
2019010209:3809:38 AM
2019010209:3909:39 AM
2019010209:4009:40 AM
2019010209:4109:41 AM
2019010209:4209:42 AM
2019010209:4309:43 AM
2019010209:4409:44 AM
2019010209:4509:45 AM
2019010209:4609:46 AM
2019010209:4709:47 AM
2019010209:4809:48 AM
2019010209:4909:49 AM
2019010209:5009:50 AM
2019010209:5109:51 AM
2019010209:5209:52 AM
2019010209:5309:53 AM
2019010209:5409:54 AM
2019010209:5509:55 AM
2019010209:5609:56 AM
2019010209:5709:57 AM
2019010209:5809:58 AM
2019010209:5909:59 AM
2019010210:0010 AM
2019010210:0110:01 AM
2019010210:0210:02 AM
2019010210:0310:03 AM
2019010210:0410:04 AM
2019010210:0510:05 AM
2019010210:0610:06 AM
2019010210:0710:07 AM
2019010210:0810:08 AM
2019010210:0910:09 AM
2019010210:1010:10 AM
2019010210:1110:11 AM
2019010210:1210:12 AM
2019010210:1310:13 AM
2019010210:1410:14 AM
2019010210:15

2019010215:513:51 PM
2019010215:523:52 PM
2019010215:533:53 PM
2019010215:543:54 PM
2019010215:553:55 PM
2019010215:563:56 PM
2019010215:573:57 PM
2019010215:583:58 PM
2019010215:593:59 PM
2019010309:3009:30 AM
2019010309:3109:31 AM
2019010309:3209:32 AM
2019010309:3309:33 AM
2019010309:3409:34 AM
2019010309:3509:35 AM
2019010309:3609:36 AM
2019010309:3709:37 AM
2019010309:3809:38 AM
2019010309:3909:39 AM
2019010309:4009:40 AM
2019010309:4109:41 AM
2019010309:4209:42 AM
2019010309:4309:43 AM
2019010309:4409:44 AM
2019010309:4509:45 AM
2019010309:4609:46 AM
2019010309:4709:47 AM
2019010309:4809:48 AM
2019010309:4909:49 AM
2019010309:5009:50 AM
2019010309:5109:51 AM
2019010309:5209:52 AM
2019010309:5309:53 AM
2019010309:5409:54 AM
2019010309:5509:55 AM
2019010309:5609:56 AM
2019010309:5709:57 AM
2019010309:5809:58 AM
2019010309:5909:59 AM
2019010310:0010 AM
2019010310:0110:01 AM
2019010310:0210:02 AM
2019010310:0310:03 AM
2019010310:0410:04 AM
2019010310:0510:05 AM
2019010310:0610:06 AM


2019010315:423:42 PM
2019010315:433:43 PM
2019010315:443:44 PM
2019010315:453:45 PM
2019010315:463:46 PM
2019010315:473:47 PM
2019010315:483:48 PM
2019010315:493:49 PM
2019010315:503:50 PM
2019010315:513:51 PM
2019010315:523:52 PM
2019010315:533:53 PM
2019010315:543:54 PM
2019010315:553:55 PM
2019010315:563:56 PM
2019010315:573:57 PM
2019010315:583:58 PM
2019010315:593:59 PM
2019010409:3009:30 AM
2019010409:3109:31 AM
2019010409:3209:32 AM
2019010409:3309:33 AM
2019010409:3409:34 AM
2019010409:3509:35 AM
2019010409:3609:36 AM
2019010409:3709:37 AM
2019010409:3809:38 AM
2019010409:3909:39 AM
2019010409:4009:40 AM
2019010409:4109:41 AM
2019010409:4209:42 AM
2019010409:4309:43 AM
2019010409:4409:44 AM
2019010409:4509:45 AM
2019010409:4609:46 AM
2019010409:4709:47 AM
2019010409:4809:48 AM
2019010409:4909:49 AM
2019010409:5009:50 AM
2019010409:5109:51 AM
2019010409:5209:52 AM
2019010409:5309:53 AM
2019010409:5409:54 AM
2019010409:5509:55 AM
2019010409:5609:56 AM
2019010409:5709:57 AM
201901

2019010415:333:33 PM
2019010415:343:34 PM
2019010415:353:35 PM
2019010415:363:36 PM
2019010415:373:37 PM
2019010415:383:38 PM
2019010415:393:39 PM
2019010415:403:40 PM
2019010415:413:41 PM
2019010415:423:42 PM
2019010415:433:43 PM
2019010415:443:44 PM
2019010415:453:45 PM
2019010415:463:46 PM
2019010415:473:47 PM
2019010415:483:48 PM
2019010415:493:49 PM
2019010415:503:50 PM
2019010415:513:51 PM
2019010415:523:52 PM
2019010415:533:53 PM
2019010415:543:54 PM
2019010415:553:55 PM
2019010415:563:56 PM
2019010415:573:57 PM
2019010415:583:58 PM
2019010415:593:59 PM
2019010709:3009:30 AM
2019010709:3109:31 AM
2019010709:3209:32 AM
2019010709:3309:33 AM
2019010709:3409:34 AM
2019010709:3509:35 AM
2019010709:3609:36 AM
2019010709:3709:37 AM
2019010709:3809:38 AM
2019010709:3909:39 AM
2019010709:4009:40 AM
2019010709:4109:41 AM
2019010709:4209:42 AM
2019010709:4309:43 AM
2019010709:4409:44 AM
2019010709:4509:45 AM
2019010709:4609:46 AM
2019010709:4709:47 AM
2019010709:4809:48 AM
2019010709:4909

2019010715:243:24 PM
2019010715:253:25 PM
2019010715:263:26 PM
2019010715:273:27 PM
2019010715:283:28 PM
2019010715:293:29 PM
2019010715:303:30 PM
2019010715:313:31 PM
2019010715:323:32 PM
2019010715:333:33 PM
2019010715:343:34 PM
2019010715:353:35 PM
2019010715:363:36 PM
2019010715:373:37 PM
2019010715:383:38 PM
2019010715:393:39 PM
2019010715:403:40 PM
2019010715:413:41 PM
2019010715:423:42 PM
2019010715:433:43 PM
2019010715:443:44 PM
2019010715:453:45 PM
2019010715:463:46 PM
2019010715:473:47 PM
2019010715:483:48 PM
2019010715:493:49 PM
2019010715:503:50 PM
2019010715:513:51 PM
2019010715:523:52 PM
2019010715:533:53 PM
2019010715:543:54 PM
2019010715:553:55 PM
2019010715:563:56 PM
2019010715:573:57 PM
2019010715:583:58 PM
2019010715:593:59 PM
2019010809:3009:30 AM
2019010809:3109:31 AM
2019010809:3209:32 AM
2019010809:3309:33 AM
2019010809:3409:34 AM
2019010809:3509:35 AM
2019010809:3609:36 AM
2019010809:3709:37 AM
2019010809:3809:38 AM
2019010809:3909:39 AM
2019010809:4009:40 AM
20

...

2019032914:022:02 PM
2019032914:032:03 PM
2019032914:042:04 PM
2019032914:052:05 PM
2019032914:062:06 PM
2019032914:072:07 PM
2019032914:082:08 PM
2019032914:092:09 PM
2019032914:102:10 PM
2019032914:112:11 PM
2019032914:122:12 PM
2019032914:132:13 PM
2019032914:142:14 PM
2019032914:152:15 PM
2019032914:162:16 PM
2019032914:172:17 PM
2019032914:182:18 PM
2019032914:192:19 PM
2019032914:202:20 PM
2019032914:212:21 PM
2019032914:222:22 PM
2019032914:232:23 PM
2019032914:242:24 PM
2019032914:252:25 PM
2019032914:262:26 PM
2019032914:272:27 PM
2019032914:282:28 PM
2019032914:292:29 PM
2019032914:302:30 PM
2019032914:312:31 PM
2019032914:322:32 PM
2019032914:332:33 PM
2019032914:342:34 PM
2019032914:352:35 PM
2019032914:362:36 PM
2019032914:372:37 PM
2019032914:382:38 PM
2019032914:392:39 PM
2019032914:402:40 PM
2019032914:412:41 PM
2019032914:422:42 PM
2019032914:432:43 PM
2019032914:442:44 PM
2019032914:452:45 PM
2019032914:462:46 PM
2019032914:472:47 PM
2019032914:482:48 PM
2019032914:49

2019040910:3710:37 AM
2019040910:3810:38 AM
2019040910:3910:39 AM
2019040910:4010:40 AM
2019040910:4110:41 AM
2019040910:4210:42 AM
2019040910:4310:43 AM
2019040910:4410:44 AM
2019040910:4510:45 AM
2019040910:4610:46 AM
2019040910:4710:47 AM
2019040910:4810:48 AM
2019040910:4910:49 AM
2019040910:5010:50 AM
2019040910:5110:51 AM
2019040910:5210:52 AM
2019040910:5310:53 AM
2019040910:5410:54 AM
2019040910:5510:55 AM
2019040910:5610:56 AM
2019040910:5710:57 AM
2019040910:5810:58 AM
2019040910:5910:59 AM
2019040911:0011 AM
2019040911:0111:01 AM
2019040911:0211:02 AM
2019040911:0311:03 AM
2019040911:0411:04 AM
2019040911:0511:05 AM
2019040911:0611:06 AM
2019040911:0711:07 AM
2019040911:0811:08 AM
2019040911:0911:09 AM
2019040911:1011:10 AM
2019040911:1111:11 AM
2019040911:1211:12 AM
2019040911:1311:13 AM
2019040911:1411:14 AM
2019040911:1511:15 AM
2019040911:1611:16 AM
2019040911:1711:17 AM
2019040911:1811:18 AM
2019040911:1911:19 AM
2019040911:2011:20 AM
2019040911:2111:21 AM
2019040911:22

2019041110:1910:19 AM
2019041110:2010:20 AM
2019041110:2110:21 AM
2019041110:2210:22 AM
2019041110:2310:23 AM
2019041110:2410:24 AM
2019041110:2510:25 AM
2019041110:2610:26 AM
2019041110:2710:27 AM
2019041110:2810:28 AM
2019041110:2910:29 AM
2019041110:3010:30 AM
2019041110:3110:31 AM
2019041110:3210:32 AM
2019041110:3310:33 AM
2019041110:3410:34 AM
2019041110:3510:35 AM
2019041110:3610:36 AM
2019041110:3710:37 AM
2019041110:3810:38 AM
2019041110:3910:39 AM
2019041110:4010:40 AM
2019041110:4110:41 AM
2019041110:4210:42 AM
2019041110:4310:43 AM
2019041110:4410:44 AM
2019041110:4510:45 AM
2019041110:4610:46 AM
2019041110:4710:47 AM
2019041110:4810:48 AM
2019041110:4910:49 AM
2019041110:5010:50 AM
2019041110:5110:51 AM
2019041110:5210:52 AM
2019041110:5310:53 AM
2019041110:5410:54 AM
2019041110:5510:55 AM
2019041110:5610:56 AM
2019041110:5710:57 AM
2019041110:5810:58 AM
2019041110:5910:59 AM
2019041111:0011 AM
2019041111:0111:01 AM
2019041111:0211:02 AM
2019041111:0311:03 AM
2019041111:04

2019041510:0110:01 AM
2019041510:0210:02 AM
2019041510:0310:03 AM
2019041510:0410:04 AM
2019041510:0510:05 AM
2019041510:0610:06 AM
2019041510:0710:07 AM
2019041510:0810:08 AM
2019041510:0910:09 AM
2019041510:1010:10 AM
2019041510:1110:11 AM
2019041510:1210:12 AM
2019041510:1310:13 AM
2019041510:1410:14 AM
2019041510:1510:15 AM
2019041510:1610:16 AM
2019041510:1710:17 AM
2019041510:1810:18 AM
2019041510:1910:19 AM
2019041510:2010:20 AM
2019041510:2110:21 AM
2019041510:2210:22 AM
2019041510:2310:23 AM
2019041510:2410:24 AM
2019041510:2510:25 AM
2019041510:2610:26 AM
2019041510:2710:27 AM
2019041510:2810:28 AM
2019041510:2910:29 AM
2019041510:3010:30 AM
2019041510:3110:31 AM
2019041510:3210:32 AM
2019041510:3310:33 AM
2019041510:3410:34 AM
2019041510:3510:35 AM
2019041510:3610:36 AM
2019041510:3710:37 AM
2019041510:3810:38 AM
2019041510:3910:39 AM
2019041510:4010:40 AM
2019041510:4110:41 AM
2019041510:4210:42 AM
2019041510:4310:43 AM
2019041510:4410:44 AM
2019041510:4510:45 AM
2019041510

C:\Users\Vladimir Dyagilev\Anaconda3\lib\site-packages\ipykernel_launcher.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  


### Add Technical Indicators as features

In [5]:
price_df = dataset_df.open
volume_df = dataset_df.volume

ema10_list = ema(price_df.tolist(), n=10)
ema20_list = ema(price_df.tolist(), n=20)
ema50_list = ema(price_df.tolist(), n=20)

sma10_list = sma(price_df.tolist(), n=20)
sma20_list = sma(price_df.tolist(), n=20)
sma50_list = sma(price_df.tolist(), n=20)

bb10 = bollinger_bands(price_df.tolist(), 10, mult=2)
bb10_low = [x[0] for x in bb10]
bb10_mid = [x[1] for x in bb10]
bb10_up = [x[2] for x in bb10]

bb20 = bollinger_bands(price_df.tolist(), 20, mult=2)
bb20_low = [x[0] for x in bb20]
bb20_mid = [x[1] for x in bb20]
bb20_up = [x[2] for x in bb20]

macd_list = macd(price_df.tolist())

obv_list = obv(volume_df.tolist(), price_df.tolist())

### We have indicators for a recent slice of data, so we cut out previous candlesticks

In [6]:
ti_dict = {'ema10': ema10_list, 'ema20': ema20_list, 'ema50': ema50_list, 'sma10': sma10_list,
          'sma20': sma20_list, 'sma50': sma50_list, 'macd': macd_list, 'obv': obv_list,
          'bb10_low': bb10_low, 'bb10_mid': bb10_mid, 'bb10_up': bb10_up, 'bb20_low': bb20_low,
          'bb20_mid': bb20_mid, 'bb20_up': bb20_up}
l = ''
c = 100 ** 10
for label, data in ti_dict.items():
    if len(data) < c:
        l, c = label, len(data)
l, c

('macd', 17438)

In [7]:
# First cut down our TI's
for label, data in ti_dict.items():
    cut_amount = len(data) - len(macd_list)
    ti_dict[label] = data[cut_amount:]    

In [8]:
# Next cut down our dataset_df
dataset_df = dataset_df.iloc[dataset_df.shape[0]-len(macd_list):]
dataset_df.shape[0]

17438

In [9]:
for label, data in ti_dict.items():
    dataset_df[label] = data
dataset_df.head()

,open_time,open,high,low,close,volume,ema10,ema20,ema50,sma10,sma20,sma50,macd,obv,bb10_low,bb10_mid,bb10_up,bb20_low,bb20_mid,bb20_up
26,1501639200000,0.083030,0.083537,0.082225,0.083236,296.728,0.443623,0.483246,0.483246,0.080470,0.080470,0.080470,0.342720,3009.083,0.082662,0.085992,0.079332,0.080470,0.086044,0.074896
27,1501642800000,0.083236,0.083276,0.082689,0.083271,396.149,0.446200,0.520459,0.520459,0.080812,0.080812,0.080812,0.277345,3405.232,0.083066,0.085321,0.080811,0.080812,0.086125,0.075499
28,1501646400000,0.083191,0.083276,0.082163,0.082594,501.075,0.448264,0.554082,0.554082,0.081090,0.081090,0.081090,0.216324,2904.157,0.083321,0.084929,0.081712,0.081090,0.086298,0.075883
29,1501650000000,0.082453,0.082928,0.082192,0.082425,453.661,0.449215,0.583766,0.583766,0.081353,0.081353,0.081353,0.159415,2450.496,0.083512,0.084398,0.082625,0.081353,0.086427,0.076279
30,1501653600000,0.082327,0.082941,0.081902,0.082113,484.671,0.449866,0.610496,0.610496,0.081500,0.081500,0.081500,0.106433,1965.825,0.083429,0.084517,0.082341,0.081500,0.086522,0.076478


### Add previous candle data as features

In [25]:
# Add columns with default values into the DataFrame
features = ['prev_high', 'prev2_high', 'prev3_high', 'prev4_high', 'prev5_high', 'prev6_high',
            'prev7_high', 'prev8_high', 'prev9_high' ,'prev10_high', 'prev_low', 'prev2_low', 
            'prev3_low', 'prev4_low', 'prev5_low', 'prev6_low', 'prev7_low', 'prev8_low',
            'prev9_low', 'prev10_low', 
            'prev_open', 'prev2_open', 'prev3_open', 'prev4_open', 'prev5_open', 'prev6_open',
            'prev7_open', 'prev8_open', 'prev9_open' ,'prev10_open']

for f in features:
    dataset_df[f] = 'default'
    
new_features = []
    
# Add the features one by one
for feature in features:
    num = feature[4] # Get the number
    if num == '1': num = 10 
    if num == '_': num = 1
    num = int(num)
    
    if 'high' in feature:
        t = 'high'
    elif 'low' in feature:
        t = 'low'
    elif 'open' in feature:
        t = 'open'

    temp = []
    for i in tqdm(range(dataset_df.shape[0])):
        if i >= num:
            temp.append(dataset_df.iloc[i-num][t])
            
    new_features.append(temp)
            

### Once again cut down our candlesticks, this time for the OPHCV data

In [26]:
x = len(new_features[0])
# determine shortest length
for feature in new_features:
    if len(feature) < x:
        x = len(feature)
x        

17428

In [27]:
# Cut down all datasets
for index, feature in enumerate(new_features):
    new_features[index] = feature[len(feature)-x:]

In [31]:
# Next cut down our dataset_df
dataset_df = dataset_df.iloc[dataset_df.shape[0]-x:]
dataset_df.shape[0]

17428

In [32]:
# Add the new columns into the dataset
for idx, feature in enumerate(features):
    dataset_df[feature] = new_features[idx]

In [39]:
print(dataset_df.shape)
dataset_df.to_csv('training_data.csv')

dataset_df.head()

(17428, 50)


,open_time,open,high,low,close,volume,ema10,ema20,ema50,sma10,...,prev_open,prev2_open,prev3_open,prev4_open,prev5_open,prev6_open,prev7_open,prev8_open,prev9_open,prev10_open
36,1501675200000,0.080901,0.081020,0.079679,0.079893,622.040,0.449574,0.722661,0.722661,0.082563,...,0.081359,0.081898,0.083004,0.082033,0.082113,0.082327,0.082453,0.083191,0.083236,0.083030
37,1501678800000,0.079893,0.082663,0.079893,0.082294,645.395,0.447726,0.733729,0.733729,0.082659,...,0.080901,0.081359,0.081898,0.083004,0.082033,0.082113,0.082327,0.082453,0.083191,0.083236
38,1501682400000,0.082294,0.082563,0.080099,0.080960,423.548,0.448616,0.746144,0.746144,0.082619,...,0.079893,0.080901,0.081359,0.081898,0.083004,0.082033,0.082113,0.082327,0.082453,0.083191
39,1501686000000,0.080988,0.081093,0.080187,0.080383,575.297,0.448037,0.756071,0.756071,0.082670,...,0.082294,0.079893,0.080901,0.081359,0.081898,0.083004,0.082033,0.082113,0.082327,0.082453
40,1501689600000,0.080383,0.081341,0.080244,0.081341,492.436,0.446959,0.764447,0.764447,0.082555,...,0.080988,0.082294,0.079893,0.080901,0.081359,0.081898,0.083004,0.082033,0.082113,0.082327


### Add our dependant variable into the dataset - the trading signal

### Train the model

In [37]:
procs = [FillMissing, Categorify, Normalize]
valid_idx = range(int(0.7 * dataset_df.shape[0]), len(dataset_df))
valid_idx

range(12199, 17428)

In [ ]:
dep_var = ''